In [94]:
import numpy as np
from scipy.special import jv
import plotly.graph_objects as go
from plotly.subplots import make_subplots

TRANSMITTER_POWER_W = 12.0 # High power mode
# TRANSMITTER_POWER_W = 2.0 # Low power mode
TX_ANT_GAIN_DBI = -1.25
# TX_ANT_GAIN_DBI = 3.30
# TX_ANT_GAIN_DBI = 0.00

TX_CABLE_LOSS_DB = 2.75 # Good
FREQ_MHZ = 2250 # Good
L_ATM = 0.19 # Good
L_POL = 0.22 # Good
RX_GT_DB_K = 33.63
K_BOLTZMANN_DBW = -228.599167  # 10*log10(1.380649e-23)
RX_SYSTEM_LOSS_DB = 0.6
CARRIER_LOOP_BW_HZ = 45.0      # 2B0 (Carrier Loop Noise Bandwidth)
SYMBOL_RATE_SPS = 1187.84 * 1000 # Specific to 1024 kbps mode
MOD_INDEX_RAD = 1.25           # theta_1 (Data)
RNG_MOD_INDEX = 0.176          # theta_2 (Ranging)
CMD_MOD_INDEX = 0.236          # theta_3 (Command turn-around)
REQUIRED_CARRIER_SNR = 10.0
REQUIRED_EBNO = 2.55
DSN_ANT_GAIN_DBI = 55.93       # DSS27 Ground Gain
DSN_MISC_LOSS_DB = 0.10        # Coupling/Misc loss in Ground Rx
BWG_ANT_GAIN_DBI= 56.8         # 34m BWG Antenna Gain


def calculate_margins(altitudes, rate, mode):
    "Calculate Link Margins for given altitudes, data rate, and power mode"

    # Carrier Suppression: Pc/Pt = cos^2(t1) * J0^2(t2) * J0^2(t3)
    sup_c = 10 * (np.log10(np.cos(MOD_INDEX_RAD)**2) + 
                  np.log10(jv(0, RNG_MOD_INDEX)**2) + 
                  np.log10(jv(0, CMD_MOD_INDEX)**2))

    # TLM Suppression: Pd/Pt = sin^2(t1) * J0^2(t2) * J0^2(t3)
    sup_tlm = 10 * (np.log10(np.sin(MOD_INDEX_RAD)**2) +
                    np.log10(jv(0, RNG_MOD_INDEX)**2) +
                    np.log10(jv(0, CMD_MOD_INDEX)**2))

    # Ground AGC Receiver Power (dBm)
    eirp = (10 * np.log10(2 if mode == "Low Power" else 12)) - TX_CABLE_LOSS_DB + TX_ANT_GAIN_DBI
    path_loss = 32.45 + (20 * (np.log10(FREQ_MHZ) + np.log10(altitudes)))
    rcv_iso_pwr_dbw = eirp - path_loss - L_ATM - L_POL
    dsn_rcvr_agc_dbm = rcv_iso_pwr_dbw + 30 + DSN_ANT_GAIN_DBI - DSN_MISC_LOSS_DB + sup_c

    # Eb/No Margin
    recieved_pr_no= rcv_iso_pwr_dbw + 34.83 - K_BOLTZMANN_DBW
    playback= 10 * np.log10(rate * 1000)
    pd_no = recieved_pr_no + sup_tlm
    eb_no_net = pd_no - RX_SYSTEM_LOSS_DB - playback
    eb_no_margin = eb_no_net - REQUIRED_EBNO

    # Receiver Power (dBm)
    rcvr_pwr_dbm= rcv_iso_pwr_dbw + BWG_ANT_GAIN_DBI + 30

    return eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm


def add_plot_data(fig):
    "build the RF link margin plot"

    altitudes_km = np.linspace(500, 160000, 10000)

    # Add Eb/No values
    # (1024)
    eb_no_margin, x, y = calculate_margins(altitudes_km, 1024, "High Power")
    eb_no_margin_ref, x, y = calculate_margins(140000, 1024, "High Power")
    fig.add_trace(go.Scatter(x=altitudes_km, y=eb_no_margin,
                            name= f'{1024} kbps, High Power', mode='lines+markers',
                            line=dict(color='green', width=2)), row= 1, col=1)
    fig.add_annotation(x=140000, y= eb_no_margin_ref, text= f"{eb_no_margin_ref:.2f} dB<br>1024 kbps, High Power",
                       showarrow=True, font= dict(color= "white", size=14))

    # (512, High Power)
    eb_no_margin, x, y = calculate_margins(altitudes_km, 512, "High Power")
    eb_no_margin_ref, x, y = calculate_margins(140000, 512, "High Power")
    fig.add_trace(go.Scatter(x=altitudes_km, y=eb_no_margin,
                            name= f'{512} kbps, High Power', mode='lines+markers',
                            line=dict(color='blue', width=2)), row= 1, col=1)
    fig.add_annotation(x=140000, y= eb_no_margin_ref, text= f"{eb_no_margin_ref:.2f} dB<br>512 kbps, High Power",
                       showarrow=True, font= dict(color= "white", size=14))

    # (128, Low Power)
    eb_no_margin, x, y = calculate_margins(altitudes_km, 128, "Low Power")
    eb_no_margin_ref, x, y = calculate_margins(140000, 128, "Low Power")
    fig.add_trace(go.Scatter(x=altitudes_km, y=eb_no_margin,
                            name= f'{128} kbps, Low Power', mode='lines+markers',
                            line=dict(color='red', width=2)), row= 1, col=1)
    fig.add_annotation(x=140000, y= eb_no_margin_ref, text= f"{eb_no_margin_ref:.2f} dB<br>128 kbps, Low Power",
                    showarrow=True, font= dict(color= "white", size=14))

    return fig


def format_plot(fig):
    "Final formatting of the plot object after generation"
        # Layout adjustments
    fig.update_layout(
        title= dict(text= f"AXAF Link Budget Analysis (34M DSS27, 1024, 512, 128 kbps)",
                    font= dict(color= "#FFFFFF")),
        paper_bgcolor= "#202020", plot_bgcolor= "#000000",
        )

    # X-Axis
    getattr(fig.layout, "xaxis").title.text= "Slant Range (km)"
    getattr(fig.layout, "xaxis").color= "#FFFFFF"
    getattr(fig.layout, "xaxis").range= [60000, 150000]

    # Y-Axis
    getattr(fig.layout, "yaxis").title.text= "Eb/No Margin (dB)"
    getattr(fig.layout, "yaxis").title.font.color= "#FFFFFF"
    getattr(fig.layout, "yaxis").color= "#FFFFFF"
    getattr(fig.layout, "yaxis").range= [-5, 15]


def main():
    # Init plotly object
    fig = make_subplots(rows= 1, cols= 1, shared_xaxes= True,
                        vertical_spacing= 0.04,
                        specs= [[{"secondary_y": False}]])

    # for data_rate in DATA_RATE_BPS:
    add_plot_data(fig)
    format_plot(fig)

    fig.add_vline(x=140000, line_dash="dash", line_color="red", opacity=0.7, line_width= 7)
    fig.add_hrect(y0= 0, y1=-5, fillcolor= "LightSalmon", layer= "below", opacity=0.3,
                  secondary_y= False, line_width= 2)
    fig.add_annotation(x=140000, y=20, text= "Max<br>Apogee",
                    showarrow=False, row= 1, col= 1, font= dict(color= "#000000", size= 28),
                    secondary_y= False, bgcolor= "#FFFFFF", bordercolor= "#585858", borderwidth= 2)
    fig.add_annotation(x=105000, y=-3, text= "No Link Margin Zone<br>(Below 0 dB)",
                showarrow=False, row= 1, col= 1, font= dict(color= "#000000", size= 28),
                secondary_y= False, bgcolor= "#FFFFFF", bordercolor= "#585858", borderwidth= 2)

    fig.update_xaxes(
            showspikes=True, spikemode='across', spikesnap='cursor',
            spikethickness=1, spikecolor="white", spikedash="solid",
            matches='x')

    fig.update_layout(
        hovermode='x', spikedistance=-1, hoverdistance=-1,
        paper_bgcolor="#202020", plot_bgcolor="#000000",
        height=1000,  width=1500, showlegend= True,
        legend=dict(title= "Eb/No Margins (dB)", font= dict(size=12, color="black"), 
                    bgcolor= "#FFFFFF", bordercolor= "#000000", borderwidth= 1))

    fig.show()


main()


In [ ]:
import numpy as np
from scipy.special import jv
import plotly.graph_objects as go
from plotly.subplots import make_subplots

TX_CABLE_LOSS_DB = 2.75 # Good
FREQ_MHZ = 2250 # Good
L_ATM = 0.19 # Good
L_POL = 0.22 # Good
RX_GT_DB_K = 33.63
K_BOLTZMANN_DBW = -228.599167  # 10*log10(1.380649e-23)
RX_SYSTEM_LOSS_DB = 0.6
CARRIER_LOOP_BW_HZ = 45.0      # 2B0 (Carrier Loop Noise Bandwidth)
SYMBOL_RATE_SPS = 1187.84 * 1000 # Specific to 1024 kbps mode
MOD_INDEX_RAD = 1.25           # theta_1 (Data)
RNG_MOD_INDEX = 0.176          # theta_2 (Ranging)
CMD_MOD_INDEX = 0.236          # theta_3 (Command turn-around)
REQUIRED_CARRIER_SNR = 10.0
REQUIRED_EBNO = 2.55
DSN_ANT_GAIN_DBI = 55.93       # DSS27 Ground Gain
DSN_MISC_LOSS_DB = 0.10        # Coupling/Misc loss in Ground Rx
BWG_ANT_GAIN_DBI= 56.8         # 34m BWG Antenna Gain


def calculate_margins(altitudes, rate, mode, gain):
    "Calculate Link Margins for given altitudes, data rate, and power mode"

    # Carrier Suppression: Pc/Pt = cos^2(t1) * J0^2(t2) * J0^2(t3)
    sup_c = 10 * (np.log10(np.cos(MOD_INDEX_RAD)**2) + 
                  np.log10(jv(0, RNG_MOD_INDEX)**2) + 
                  np.log10(jv(0, CMD_MOD_INDEX)**2))

    # TLM Suppression: Pd/Pt = sin^2(t1) * J0^2(t2) * J0^2(t3)
    sup_tlm = 10 * (np.log10(np.sin(MOD_INDEX_RAD)**2) +
                    np.log10(jv(0, RNG_MOD_INDEX)**2) +
                    np.log10(jv(0, CMD_MOD_INDEX)**2))

    # Ground AGC Receiver Power (dBm)
    eirp = (10 * np.log10(2 if mode == "Low Power" else 12)) - TX_CABLE_LOSS_DB + gain
    path_loss = 32.45 + (20 * (np.log10(FREQ_MHZ) + np.log10(altitudes)))
    rcv_iso_pwr_dbw = eirp - path_loss - L_ATM - L_POL
    dsn_rcvr_agc_dbm = rcv_iso_pwr_dbw + 30 + DSN_ANT_GAIN_DBI - DSN_MISC_LOSS_DB + sup_c

    # Eb/No Margin
    recieved_pr_no= rcv_iso_pwr_dbw + 34.83 - K_BOLTZMANN_DBW
    playback= 10 * np.log10(rate * 1000)
    pd_no = recieved_pr_no + sup_tlm
    eb_no_net = pd_no - RX_SYSTEM_LOSS_DB - playback
    eb_no_margin = eb_no_net - REQUIRED_EBNO

    # Receiver Power (dBm)
    rcvr_pwr_dbm= rcv_iso_pwr_dbw + BWG_ANT_GAIN_DBI + 30

    return eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm


def add_plot_data(fig):
    "build the RF link margin plot"

    altitudes_km = np.linspace(500, 160000, 10000)

    # Add Eb/No values
    # (High Power, Tx Gain 3.30 dBi)
    eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm = calculate_margins(altitudes_km, 1024, "High Power", 3.30)
    fig.add_trace(go.Scatter(x=altitudes_km, y=rcvr_pwr_dbm,
                            name= "High Power (Peak)<br>3.30 dBi Tx Gain", mode='lines+markers',
                            line= dict(color='green', width=2, dash= "solid")), row= 1, col=1)

    # (High Power, Tx Gain 0.00 dBi)
    eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm = calculate_margins(altitudes_km, 1024, "High Power", 0.00)
    fig.add_trace(go.Scatter(x=altitudes_km, y=rcvr_pwr_dbm,
                            name= "High Power (Typical)<br>0.00 dBi Tx Gain", mode='lines',
                            line= dict(color='green', width=2, dash= "dot")), row= 1, col=1)

    # (High Power, Tx Gain -1.25 dBi)
    eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm = calculate_margins(altitudes_km, 1024, "High Power", -1.25)
    fig.add_trace(go.Scatter(x=altitudes_km, y=rcvr_pwr_dbm,
                            name= "High Power (Min)<br>-1.25 dBi Tx Gain", mode='lines',
                            line= dict(color='green', width=2, dash= "longdash")), row= 1, col=1)

    # (Low Power, Tx Gain 3.30 dBi)
    eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm = calculate_margins(altitudes_km, 1024, "Low Power", 3.30)
    fig.add_trace(go.Scatter(x=altitudes_km, y=rcvr_pwr_dbm,
                            name= "Low Power (Peak)<br>3.30 dBi Tx Gain", mode='lines+markers',
                            line= dict(color='blue', width=2, dash= "solid")), row= 1, col=1)

    # (Low Power, Tx Gain 0.00 dBi)
    eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm = calculate_margins(altitudes_km, 1024, "Low Power", 0.00)
    fig.add_trace(go.Scatter(x=altitudes_km, y=rcvr_pwr_dbm,
                            name= "Low Power (Typical)<br>0.00 dBi Tx Gain", mode='lines',
                            line= dict(color='blue', width=2, dash= "dot")), row= 1, col=1)

    # (Low Power, Tx Gain -1.25 dBi)
    eb_no_margin, dsn_rcvr_agc_dbm, rcvr_pwr_dbm = calculate_margins(altitudes_km, 1024, "Low Power", -1.25)
    fig.add_trace(go.Scatter(x=altitudes_km, y=rcvr_pwr_dbm,
                            name= "Low Power (Min)<br>-1.25 dBi Tx Gain", mode='lines',
                            line= dict(color='blue', width=2, dash= "longdash")), row= 1, col=1)

    # Margin Line
    fig.add_hline(y= -77-1.8, line_dash="dash", line_color="green", opacity=0.7, line_width= 3, row= 1, col=1)
    fig.add_annotation(x=17000, y=-77-1.8, text= "1.8 dB Margin",
                       showarrow=False, row= 1, col= 1, font= dict(color= "#000000", size= 28),
                       secondary_y= False, bgcolor= "#FFFFFF", bordercolor= "#585858", borderwidth= 2)

    # DSN Hazard Box
    fig.add_hrect(y0= -77, y1=-60, fillcolor= "LightSalmon", layer= "below", opacity=0.3,
                  secondary_y= False, line_width= 2)
    fig.add_annotation(x=10000, y= -67, text= "DSN Hazard Zone<br>(Above -77 dBm)",
                       showarrow=False, row= 1, col= 1, font= dict(color= "#FFFFFF", size= 28),
                       secondary_y= False)

    return fig


def format_plot(fig):
    "Final formatting of the plot object after generation"
        # Layout adjustments
    fig.update_layout(
        title= dict(text= f"AXAF Link Budget Analysis (34M DSS27, 1024, 512, 128 kbps)",
                    font= dict(color= "#FFFFFF")),
        paper_bgcolor= "#202020", plot_bgcolor= "#000000",
        )

    # X-Axis
    getattr(fig.layout, "xaxis").title.text= "Altitude (km)"
    getattr(fig.layout, "xaxis").color= "#FFFFFF"
    getattr(fig.layout, "xaxis").range= [500, 20000]

    # Y-Axis
    getattr(fig.layout, "yaxis").title.text= "DSN BWG Receiver Power (dBm)"
    getattr(fig.layout, "yaxis").title.font.color= "#FFFFFF"
    getattr(fig.layout, "yaxis").color= "#FFFFFF"
    getattr(fig.layout, "yaxis").range= [-100, -60]


def main():
    # Init plotly object
    fig = make_subplots(rows= 1, cols= 1, shared_xaxes= True,
                        vertical_spacing= 0.04,
                        specs= [[{"secondary_y": False}]])

    # for data_rate in DATA_RATE_BPS:
    add_plot_data(fig)
    format_plot(fig)

    fig.update_xaxes(
            showspikes=True, spikemode='across', spikesnap='cursor',
            spikethickness=1, spikecolor="white", spikedash="solid",
            matches='x')

    fig.update_layout(
        hovermode='x', spikedistance=-1, hoverdistance=-1,
        paper_bgcolor="#202020", plot_bgcolor="#000000",
        height=1000,  width=1500, showlegend= True,
        legend=dict(title= "Power Modes", font= dict(size=12, color="black"), 
                    bgcolor= "#FFFFFF", bordercolor= "#000000", borderwidth= 1))

    fig.show()
    # fig.write_html("AXAF RF Link Budget.html")


main()


In [82]:
%reset -f